<a href="https://colab.research.google.com/github/afifahnisa17/machine_learning/blob/main/UTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LOAD DATA

In [4]:
!pip install annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 16.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp312-cp312-linux_x86_64.whl size=551807 sha256=d9ae6387bee484d93466ecbb3a986e8f07b9cf758afb9c03f98509fa78d745f1
  Stored in directory: /root/.cache/pip/wheels/db/b9/53/a3b2d1fe1743abadddec6aa541294b24fdbc39d7800bc57311
Successfully built annoy


In [5]:
# Import library yang diperlukan
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, davies_bouldin_score
from annoy import AnnoyIndex
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Machine Learning/UTS/UCI_Credit_Card.csv')

df.info()
df.describe()
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID                          30000 non-null  int64  
 1   LIMIT_BAL                   30000 non-null  float64
 2   SEX                         30000 non-null  int64  
 3   EDUCATION                   30000 non-null  int64  
 4   MARRIAGE                    30000 non-null  int64  
 5   AGE                         30000 non-null  int64  
 6   PAY_0                       30000 non-null  int64  
 7   PAY_2                       30000 non-null  int64  
 8   PAY_3                       30000 non-null  int64  
 9   PAY_4                       30000 non-null  int64  
 10  PAY_5                       30000 non-null  int64  
 11  PAY_6                       30000 non-null  int64  
 12  BILL_AMT1                   30000 non-null  float64
 13  BILL_AMT2                   300

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


# PRE - PROCESSING DATA

In [10]:
# ==============================
# 📦 PREPROCESSING DATA
# ==============================
import pandas as pd
from sklearn.preprocessing import StandardScaler

# --- Baca dataset ---
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Machine Learning/UTS/UCI_Credit_Card.csv")

# --- Informasi umum dataset ---
print("Jumlah baris dan kolom:", df.shape)
print("\nLima baris pertama:\n", df.head())

# --- Cek missing values ---
print("\nJumlah missing values per kolom:")
print(df.isnull().sum())

# --- Tangani missing values ---
# Untuk semua kolom numerik → isi dengan median
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

# Untuk kolom kategorikal (jika ada) → isi dengan modus
cat_cols = df.select_dtypes(include=['object']).columns
for col in cat_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)

# --- Buat fitur baru: total pembayaran & total tagihan 6 bulan terakhir ---
df["TOTAL_PAY_AMT"] = df[[f"PAY_AMT{i}" for i in range(1, 7)]].sum(axis=1)
df["TOTAL_BILL_AMT"] = df[[f"BILL_AMT{i}" for i in range(1, 7)]].sum(axis=1)

# --- Fitur tambahan: rasio pembayaran terhadap tagihan total ---
df["PAY_RATIO"] = df["TOTAL_PAY_AMT"] / (df["TOTAL_BILL_AMT"] + 1)  # tambah +1 biar tidak division by zero

# --- Hilangkan kolom ID & target ---
X = df.drop(columns=["ID", "default.payment.next.month"], errors="ignore")

# --- Standarisasi semua fitur numerik ---
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# --- Informasi akhir ---
print("\nPreprocessing selesai ✅")
print("Jumlah fitur setelah ditambah:", X.shape[1])
print("Shape data setelah scaling:", X_scaled.shape)

# --- (Opsional) simpan hasil preprocessing ke file baru ---
processed_df = pd.DataFrame(X_scaled, columns=X.columns)
processed_df.to_csv("/content/drive/MyDrive/Colab Notebooks/Machine Learning/UTS/credit_card_preprocessed.csv", index=False)
print("✅ Data tersimpan: credit_card_preprocessed.csv")


Jumlah baris dan kolom: (30000, 25)

Lima baris pertama:
    ID  LIMIT_BAL  SEX  EDUCATION  MARRIAGE  AGE  PAY_0  PAY_2  PAY_3  PAY_4  \
0   1    20000.0    2          2         1   24      2      2     -1     -1   
1   2   120000.0    2          2         2   26     -1      2      0      0   
2   3    90000.0    2          2         2   34      0      0      0      0   
3   4    50000.0    2          2         1   37      0      0      0      0   
4   5    50000.0    1          2         1   57     -1      0     -1      0   

   ...  BILL_AMT4  BILL_AMT5  BILL_AMT6  PAY_AMT1  PAY_AMT2  PAY_AMT3  \
0  ...        0.0        0.0        0.0       0.0     689.0       0.0   
1  ...     3272.0     3455.0     3261.0       0.0    1000.0    1000.0   
2  ...    14331.0    14948.0    15549.0    1518.0    1500.0    1000.0   
3  ...    28314.0    28959.0    29547.0    2000.0    2019.0    1200.0   
4  ...    20940.0    19146.0    19131.0    2000.0   36681.0   10000.0   

   PAY_AMT4  PAY_AMT5  PAY_A

In [11]:
# ==============================
# 🤖 CLUSTERING
# ==============================
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score
import numpy as np

# --- Gunakan hasil dari preprocessing sebelumnya ---
# Pastikan variabel X_scaled sudah ada
print("Mulai proses clustering...")

# ==============================
# 🔹 K-MEANS CLUSTERING
# ==============================
# Tentukan jumlah cluster (misal 3)
k = 3
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans_labels = kmeans.fit_predict(X_scaled)

# Evaluasi K-Means
sil_kmeans = silhouette_score(X_scaled, kmeans_labels)
db_kmeans = davies_bouldin_score(X_scaled, kmeans_labels)

print("\n=== HASIL K-MEANS ===")
print(f"Jumlah Cluster: {k}")
print(f"Silhouette Score : {sil_kmeans:.4f}")
print(f"Davies–Bouldin Index : {db_kmeans:.4f}")

# ==============================
# 🔹 DBSCAN CLUSTERING
# ==============================
# eps: jarak maksimum antar titik agar dianggap satu cluster
# min_samples: minimal jumlah titik dalam satu cluster
dbscan = DBSCAN(eps=2.0, min_samples=10)
dbscan_labels = dbscan.fit_predict(X_scaled)

# Cek jumlah cluster (DBSCAN bisa punya label -1 untuk outlier)
n_clusters_dbscan = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
n_noise = list(dbscan_labels).count(-1)

# Evaluasi DBSCAN
# Pastikan hanya menghitung jika ada cluster > 1
if n_clusters_dbscan > 1:
    sil_dbscan = silhouette_score(X_scaled, dbscan_labels)
    db_dbscan = davies_bouldin_score(X_scaled, dbscan_labels)
else:
    sil_dbscan = np.nan
    db_dbscan = np.nan

print("\n=== HASIL DBSCAN ===")
print(f"Jumlah Cluster: {n_clusters_dbscan}")
print(f"Jumlah Noise Points: {n_noise}")
print(f"Silhouette Score : {sil_dbscan}")
print(f"Davies–Bouldin Index : {db_dbscan}")

# ==============================
# 🔹 Perbandingan Akhir
# ==============================
print("\n📊 PERBANDINGAN HASIL CLUSTERING")
print("----------------------------------")
print(f"{'Metode':<12} {'Silhouette':<15} {'Davies–Bouldin'}")
print(f"{'K-Means':<12} {sil_kmeans:<15.4f} {db_kmeans:<15.4f}")
print(f"{'DBSCAN':<12} {str(round(sil_dbscan,4)):<15} {str(round(db_dbscan,4)):<15}")


Mulai proses clustering...


KeyboardInterrupt: 